## Part VI: Natural Language Supervision

We will compare the performance of:

(a) traditional supervision - positive and negative labels on examples

(b) natural language supervision - explanations converted into LFs, which are then denoised and applied to unlabeled data to create a much larger but noisy training set

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass from Parts II and III.

In [2]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [3]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [4]:
F_train = featurizer.load_matrix(session, split=0)
F_dev   = featurizer.load_matrix(session, split=1)

In [5]:
from snorkel.annotations import load_gold_labels
L_gold_train = load_gold_labels(session, annotator_name='gold', split=0)
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

### Explanations

## (a) Traditional Supervision

In [6]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

/Users/bradenhancock/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [7]:
train_marginals = np.array(L_gold_train.todense()).reshape((L_gold_train.shape[0],))
# convert -1s to 0s; NOTE: there were already some 0's; this could cause weirdness
train_marginals[train_marginals==-1] = 0
print train_marginals.shape

(141,)


In [8]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=20)

Initialized RandomSearch search of size 20. Search space size = 125.


In [9]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=True, print_freq=25)

[1] Testing lr = 1.00e-02, l1_penalty = 1.00e-03, l2_penalty = 1.00e-04
[SparseLR] lr=0.01 l1=0.001 l2=0.0001
[SparseLR] Building model
[SparseLR] Training model  #epochs=50  batch=4
[SparseLR] Epoch 0 (0.09s)	Avg. loss=0.519005	NNZ=4527
[SparseLR] Epoch 25 (0.13s)	Avg. loss=0.066215	NNZ=4527
[SparseLR] Epoch 49 (0.16s)	Avg. loss=0.034189	NNZ=4527
[SparseLR] Training done (0.16s)
[SparseLR] Model saved. To load, use name
		SparseLR_0
[2] Testing lr = 1.00e-04, l1_penalty = 1.00e-06, l2_penalty = 1.00e-03
[SparseLR] lr=0.0001 l1=1e-06 l2=0.001
[SparseLR] Building model
[SparseLR] Training model  #epochs=50  batch=4
[SparseLR] Epoch 0 (0.08s)	Avg. loss=0.541291	NNZ=4527
[SparseLR] Epoch 25 (0.13s)	Avg. loss=0.525678	NNZ=4527
[SparseLR] Epoch 49 (0.17s)	Avg. loss=0.511329	NNZ=4527
[SparseLR] Training done (0.17s)
[3] Testing lr = 1.00e-03, l1_penalty = 1.00e-05, l2_penalty = 1.00e-05
[SparseLR] lr=0.001 l1=1e-05 l2=1e-05
[SparseLR] Building model
[SparseLR] Training model  #epochs=50  bat

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
19,0.010000,0.000010,0.000100,0.055901,0.9,0.105263
6,0.000001,0.001000,0.010000,0.055556,0.7,0.102941
11,0.001000,0.000001,0.001000,0.053763,1.0,0.102041
9,0.000001,0.000010,0.001000,0.053691,0.8,0.100629
18,0.000010,0.010000,0.000001,0.053435,0.7,0.099291
16,0.000001,0.010000,0.001000,0.050725,0.7,0.094595
3,0.001000,0.000001,0.001000,0.049505,1.0,0.094340
13,0.001000,0.000010,0.000100,0.048544,1.0,0.092593
2,0.001000,0.000010,0.000010,0.048309,1.0,0.092166
14,0.000100,0.010000,0.000010,0.048780,0.8,0.091954


In [10]:
_, _, _, _ = disc_model.score(session, F_dev, L_gold_dev)

Scores (Un-adjusted)
Pos. class accuracy: 0.9
Neg. class accuracy: 0.312
Precision            0.0559
Recall               0.9
F1                   0.105
----------------------------------------
TP: 9 | FP: 152 | TN: 69 | FN: 1

